In [4]:
import os
import nltk
import random
import nltk
from nltk.corpus import TaggedCorpusReader
from nltk.tag import HiddenMarkovModelTrainer, HiddenMarkovModelTagger
from sklearn.model_selection import train_test_split


In [2]:
input_dir = './data/'  # replace with the path to the folder containing the original CoNLL-2000 chunking files
output_dir = './tagonly/'  # replace with the path to the folder where the modified CoNLL-2000 files will be saved

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        input_path = os.path.join(input_dir, file_name)
        output_path = os.path.join(output_dir, file_name.replace('chunking', 'pos'))
        with open(input_path, 'r') as input_file, open(output_path, 'w') as output_file:
            for line in input_file:
                line = line.strip()
                if not line:
                    output_file.write('\n')  # write empty lines as they are
                else:
                    fields = line.split()
                    word = fields[0]
                    pos_tag = fields[1]
                    output_file.write(f'{word} {pos_tag}\n')
        print(f'{file_name} processed.')


test.txt processed.
train.txt processed.


In [5]:
train_file = './tagonly/train.txt'
test_file = './tagonly/test.txt'

def conll2tuple(filename):
    with open(filename, 'r') as f:
        data = f.read().strip()
    sents = data.split('\n\n')
    return [[nltk.tag.str2tuple(wordtag) for wordtag in sent.split('\n')] for sent in sents]

# Convert the train and test files to a list of (word, tag) tuples
train_sents = conll2tuple(train_file)
# test_sents = conll2tuple(test_file)

train_sents, test_sents = train_test_split(train_sents, test_size=0.2, random_state=42)

# Train an HMM on the train set
trainer = HiddenMarkovModelTrainer()
model = trainer.train_supervised(train_sents)

# Create an HMM tagger from the trained model
tagger = HiddenMarkovModelTagger(model._symbols, model._states, model._transitions, model._outputs, model._priors)

# Test the HMM tagger on the test set
accuracy = tagger.accuracy(test_sents)

print(f'Testing accuracy: {accuracy:.2%}')


Testing accuracy: 99.79%
